In [1]:
import os
import networkx as nx
import json, math, numpy as np
import networkx as nx
from collections import defaultdict, Counter
import torch
from torch_geometric.data import HeteroData
from torch_geometric.nn import RGCNConv
from torch.nn import Module, Linear
import torch.nn.functional as F
try:
    import orjson as fast_json
    json_load = fast_json.loads
except ImportError:
    json_load = json.loads

from networkx.readwrite import json_graph

In [2]:
# === CONFIG ===
file_paths = [
    '../dataset/cadets/ta1-cadets-e3-official-1.json',
    '../dataset/cadets/ta1-cadets-e3-official-1.json.1',
    '../dataset/cadets/ta1-cadets-e3-official-1.json.2',
    '../dataset/cadets/ta1-cadets-e3-official-1.json.3',
    '../dataset/cadets/ta1-cadets-e3-official-1.json.4',
]
start_ts = 1523300401023000000
end_ts   = 1523303101008000000

In [3]:
# === PREP ===
G = nx.DiGraph()
filtered_subjects = set()
filtered_events = []
referenced_uuids = set()
node_data_by_uuid = {}

In [4]:
# === PASS 1: Filter Subjects in time window ===
def filter_subjects():
    for path in file_paths:
        with open(path, 'r') as f:
            for line in f:
                try:
                    record = json_load(line)
                    datum = record.get("datum", {})
                    subj = datum.get("com.bbn.tc.schema.avro.cdm18.Subject")
                    if subj:
                        ts = subj.get("startTimestampNanos")
                        uuid = subj.get("uuid")
                        if ts and start_ts <= ts <= end_ts and uuid:
                            filtered_subjects.add(uuid)
                except Exception as e:
                    print(f"Error processing line:\n{line}\nException: {e}")

In [5]:
print("📌 [1] Filtering Subjects...")
filter_subjects()

📌 [1] Filtering Subjects...


In [6]:
def filter_events_and_uuids():
    for path in file_paths:
        with open(path, 'r') as f:
            for line in f:
                try:
                    record = json_load(line)
                    datum = record.get("datum", {})
                    event = datum.get("com.bbn.tc.schema.avro.cdm18.Event")
                    if not event:
                        continue

                    ts = event.get("timestampNanos")
                    if not (start_ts <= ts <= end_ts):
                        continue

                    subj = event.get("subject")
                    pred1 = event.get("predicateObject")
                    pred2 = event.get("predicateObject2")

                    subj_uuid = subj.get("com.bbn.tc.schema.avro.cdm18.UUID") if isinstance(subj, dict) else None
                    pred1_uuid = pred1.get("com.bbn.tc.schema.avro.cdm18.UUID") if isinstance(pred1, dict) else None
                    pred2_uuid = pred2.get("com.bbn.tc.schema.avro.cdm18.UUID") if isinstance(pred2, dict) else None

                    # Get paths safely even if they are None
                    pred1_path = None
                    pred2_path = None

                    if isinstance(event.get("predicateObjectPath"), dict):
                        pred1_path = event["predicateObjectPath"].get("string")

                    if isinstance(event.get("predicateObject2Path"), dict):
                        pred2_path = event["predicateObject2Path"].get("string")

                    properties = {}
                    if isinstance(event.get("properties"), dict):
                        properties = event["properties"].get("map", {})

                    event_dict = {
                        "subject": subj_uuid,
                        "type": event.get("type", "Unknown"),
                        "timestampNanos": ts,
                        "predicate1": pred1_uuid,
                        "predicate1_path": pred1_path,
                        "predicate2": pred2_uuid,
                        "predicate2_path": pred2_path,
                        "properties": properties
                    }

                    referenced_uuids.update(filter(None, [subj_uuid, pred1_uuid, pred2_uuid]))
                    filtered_events.append(event_dict)

                except Exception as e:
                    print(f"Error processing line:\n{line}\nException: {e}")


In [7]:
print("📌 [2] Filtering Events and UUIDs...")
filter_events_and_uuids()

📌 [2] Filtering Events and UUIDs...


In [8]:
def preload_nodes():
    all_needed_uuids = referenced_uuids.union(filtered_subjects)

    node_types = {
        "com.bbn.tc.schema.avro.cdm18.Subject": "Subject",
        "com.bbn.tc.schema.avro.cdm18.FileObject": "File",
        "com.bbn.tc.schema.avro.cdm18.NetFlowObject": "NetFlow",
        "com.bbn.tc.schema.avro.cdm18.RegistryKeyObject": "Registry",
        "com.bbn.tc.schema.avro.cdm18.MemoryObject": "Memory"
    }

    for path in file_paths:
        with open(path, 'r') as f:
            for line in f:
                try:
                    record = json_load(line)
                    datum = record.get("datum", {})

                    for key, base_type in node_types.items():
                        obj = datum.get(key)
                        if not obj:
                            continue

                        uuid = obj.get("uuid")
                        if not uuid or uuid not in all_needed_uuids:
                            continue

                        obj = dict(obj)  # shallow copy to mutate safely
                        obj.pop("uuid", None)

                        # Clean baseObject.hostId if present
                        base_obj = obj.get("baseObject")
                        if isinstance(base_obj, dict):
                            base_obj.pop("hostId", None)

                        if key == "com.bbn.tc.schema.avro.cdm18.Subject":
                            obj.pop("hostId", None)
                            obj.pop("localPrincipal", None)
                            node_type = "Process" if obj.get("type") == "SUBJECT_PROCESS" else "Thread"
                        else:
                            node_type = base_type

                        # Just cache — don't add nodes or edges yet
                        node_data_by_uuid[uuid] = {
                            "type": node_type,
                            "info": obj
                        }

                except Exception as e:
                    print(f"Error processing line:\n{line}\nException: {e}")


In [9]:
print("📌 [3] Preloading Node Data...")
preload_nodes()

📌 [3] Preloading Node Data...


In [10]:
# === PASS 4: Build Graph and retain important nodes ===
def build_graph():
    for uuid, node in node_data_by_uuid.items():
        G.add_node(uuid, **node)

    for e in filtered_events:
        source = e["subject"]
        label = e["type"].replace("EVENT_", "")
        ts = e["timestampNanos"]
        props = e["properties"]

        for target, path in [
            (e["predicate1"], e["predicate1_path"]),
            (e["predicate2"], e["predicate2_path"])
        ]:
            if not source or not target:
                continue
            if source not in G.nodes or target not in G.nodes:
                continue

            attrs = {
                "label": label,
                "timestampNanos": ts,
                **props
            }
            if isinstance(path, dict):
                path_str = path.get("string")
                if path_str and target in G.nodes:
                    G.nodes[target].setdefault("info", {})["path"] = path_str

            G.add_edge(source, target, **attrs)

    # Remove nodes that are completely unconnected and not a filtered subject
    to_remove = [n for n in G.nodes if G.degree(n) == 0 and n not in filtered_subjects]
    G.remove_nodes_from(to_remove)

In [11]:
print("📌 [4] Building Graph...")
build_graph()

📌 [4] Building Graph...


In [13]:
print(G)

DiGraph with 6651 nodes and 13443 edges


In [15]:
# === EXPORT ===
out_path = "malicious_Graph_test.json"
data = json_graph.node_link_data(G)
with open(out_path, "w") as f:
    json.dump(data, f, indent=2)

print(f"✅ Exported to {out_path} with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

✅ Exported to malicious_Graph_test.json with 6651 nodes and 13443 edges


In [17]:
def is_multi(G):
    return getattr(G, "is_multigraph", lambda: False)()

def iter_edges(G):
    """Yield (u, v, key, data) where key=None for DiGraph."""
    if is_multi(G):
        for u, v, k, d in G.edges(keys=True, data=True):
            yield u, v, k, d
    else:
        for u, v, d in G.edges(data=True):
            yield u, v, None, d

In [18]:
# --- 1. Đọc đồ thị node-link (từ code bạn xuất ra) ---
with open("malicious_Graph_test.json","r") as f:
    nl = json.load(f)
G = nx.node_link_graph(nl, directed=True, multigraph=True)

# --- 2. Chuẩn hoá loại nút & cạnh ---
node_types = sorted({G.nodes[n].get("type","Unknown") for n in G.nodes})
type2id = {t:i for i,t in enumerate(node_types)}

edge_types = sorted({d.get("label", "Unknown") for _, _, _, d in iter_edges(G)})
rel2id = {r: i for i, r in enumerate(edge_types)}

# --- 3. Tạo đặc trưng nút rẻ tiền nhưng hữu dụng ---
# one-hot theo node.type + deg + in/out deg (log1p)
N = G.number_of_nodes()
node_index = {n:i for i,n in enumerate(G.nodes)}
id2node = {i:n for n,i in node_index.items()}

X = np.zeros((N, len(node_types)+3), dtype=np.float32)
for n in G.nodes:
    i = node_index[n]
    t = G.nodes[n].get("type","Unknown")
    X[i, type2id[t]] = 1.0
    indeg = G.in_degree(n)
    outdeg = G.out_degree(n)
    X[i, len(node_types)+0] = math.log1p(indeg+outdeg)
    X[i, len(node_types)+1] = math.log1p(indeg)
    X[i, len(node_types)+2] = math.log1p(outdeg)

# --- 4. Gom cạnh theo relation để tạo HeteroData dạng RGCN (đơn kiểu nút) ---
# Ta coi tất cả nút cùng 1 kiểu "node" (homogeneous node type) nhưng cạnh có relation id
src_all, dst_all, rel_all = [], [], []
for u,v,k,d in iter_edges(G):
    src_all.append(node_index[u])
    dst_all.append(node_index[v])
    rel_all.append(rel2id[d.get("label","Unknown")])
edge_index = torch.tensor([src_all, dst_all], dtype=torch.long)
edge_type  = torch.tensor(rel_all, dtype=torch.long)

x = torch.tensor(X, dtype=torch.float)

# --- 5. Chia train/val/test ngẫu nhiên cho link prediction ---
M = edge_index.size(1)
perm = torch.randperm(M)
train_size = int(0.8*M); val_size = int(0.1*M)
train_idx = perm[:train_size]
val_idx   = perm[train_size:train_size+val_size]
test_idx  = perm[train_size+val_size:]

def split_ei(idx):
    return edge_index[:, idx], edge_type[idx]

ei_train, et_train = split_ei(train_idx)
ei_val,   et_val   = split_ei(val_idx)
ei_test,  et_test  = split_ei(test_idx)

# Negative sampler đơn giản
def negative_sample(num_neg, num_nodes):
    src = torch.randint(0, num_nodes, (num_neg,))
    dst = torch.randint(0, num_nodes, (num_neg,))
    return torch.stack([src, dst], dim=0)

In [19]:
class RGCNEncoder(Module):
    def __init__(self, in_dim, hid_dim, out_dim, num_rels):
        super().__init__()
        self.conv1 = RGCNConv(in_dim, hid_dim, num_rels, num_bases=min(16, num_rels))
        self.conv2 = RGCNConv(hid_dim, out_dim, num_rels, num_bases=min(16, num_rels))
    def forward(self, x, edge_index, edge_type):
        h = F.relu(self.conv1(x, edge_index, edge_type))
        h = self.conv2(h, edge_index, edge_type)
        return h

class DistMultDecoder(Module):
    def __init__(self, num_rels, dim):
        super().__init__()
        self.rel = torch.nn.Embedding(num_rels, dim)  # vector per relation
    def score(self, h, edge_index, edge_type):
        s = h[edge_index[0]]     # (E, d)
        o = h[edge_index[1]]     # (E, d)
        r = self.rel(edge_type)  # (E, d)
        return (s * r * o).sum(dim=-1)  # (E,)
    def forward(self, h, edge_index, edge_type):
        return self.score(h, edge_index, edge_type)

class RGCNLinkPred(Module):
    def __init__(self, in_dim, hid_dim, out_dim, num_rels):
        super().__init__()
        self.enc = RGCNEncoder(in_dim, hid_dim, out_dim, num_rels)
        self.dec = DistMultDecoder(num_rels, out_dim)
    def forward(self, x, edge_index, edge_type):
        h = self.enc(x, edge_index, edge_type)
        logits = self.dec(h, edge_index, edge_type)
        return logits, h


In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RGCNLinkPred(in_dim=x.size(1), hid_dim=64, out_dim=64, num_rels=len(rel2id)).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

x_d = x.to(device)

def bce_loss(pos_logit, neg_logit):
    pos_y = torch.ones_like(pos_logit)
    neg_y = torch.zeros_like(neg_logit)
    return F.binary_cross_entropy_with_logits(torch.cat([pos_logit, neg_logit]),
                                              torch.cat([pos_y, neg_y]))

def evaluate(ei, et):
    model.eval()
    with torch.no_grad():
        _, h = model(x_d, ei_train.to(device), et_train.to(device))  # encode trên train graph
        pos = model.dec(h, ei.to(device), et.to(device))
        neg_ei = negative_sample(ei.size(1), x.size(0)).to(device)
        neg_et = et[torch.randint(0, et.size(0), (ei.size(1),))].to(device)
        neg = model.dec(h, neg_ei, neg_et)
        # AUC
        y = torch.cat([torch.ones_like(pos), torch.zeros_like(neg)])
        s = torch.cat([pos, neg]).sigmoid()
        from sklearn.metrics import roc_auc_score
        return roc_auc_score(y.cpu().numpy(), s.cpu().numpy())

for epoch in range(1, 51):
    model.train()
    opt.zero_grad()
    logits, h = model(x_d, ei_train.to(device), et_train.to(device))
    neg_ei = negative_sample(ei_train.size(1), x.size(0)).to(device)
    neg_et = et_train[torch.randint(0, et_train.size(0), (ei_train.size(1),))].to(device)
    neg_logits = model.dec(h, neg_ei, neg_et)
    loss = bce_loss(logits, neg_logits)
    loss.backward()
    opt.step()
    if epoch % 10 == 0:
        auc_val = evaluate(ei_val, et_val)
        print(f"Epoch {epoch:02d} | loss {loss.item():.4f} | val AUC {auc_val:.3f}")

# Điểm bất thường cạnh = 1 - sigmoid(score)
model.eval()
with torch.no_grad():
    _, H = model(x_d, ei_train.to(device), et_train.to(device))
    test_pos = model.dec(H, ei_test.to(device), et_test.to(device)).sigmoid().cpu()
anomaly_edge_score = 1 - test_pos.numpy()  # càng cao càng lạ

# Điểm bất thường nút = trung bình top-k cạnh (in/out) bất thường
from collections import defaultdict
node_edge_scores = defaultdict(list)
ei_np = edge_index[:, test_idx].cpu().numpy()
for e_idx, score in enumerate(anomaly_edge_score):
    u, v = int(ei_np[0, e_idx]), int(ei_np[1, e_idx])
    node_edge_scores[u].append(score); node_edge_scores[v].append(score)

node_anomaly = np.zeros(N, dtype=np.float32)
for i in range(N):
    if node_edge_scores[i]:
        vals = sorted(node_edge_scores[i], reverse=True)
        k = max(1, len(vals)//5)  # top-20%
        node_anomaly[i] = float(np.mean(vals[:k]))


Epoch 10 | loss 0.4432 | val AUC 0.898
Epoch 20 | loss 0.2529 | val AUC 0.933
Epoch 30 | loss 0.0922 | val AUC 0.946
Epoch 40 | loss 0.0715 | val AUC 0.945
Epoch 50 | loss 0.0590 | val AUC 0.956
